# SQLAlchemy and Object Relational Model (ORMs)

## Motivation

SQL-based relational databases are a great way to store and query relational data. SQL database systems (e.g. Postgres) are optimized for fast query of data and efficient storage, via a variety of techniques such as indexing or compression.

SQL provides us with a descriptive language that allows us to create and modify schemas (e.g. the structure of tables and their relations), as well as inserting, updating, and querying data. SQL is a domain specific language. It does not have any programming capabilities beyond database functionalities.

Database systems are commonly used as a backend data store for applications, especially web and mobile applications. Large applications of this sort are written in a variety of layers and languages. Common backends languages including Python and node.js.

This poses a challenge. Backends written in Python or similar languages must be able to interact with database systems through SQL. Legacy designs suggest embedding SQL commands into the backend in a naive way: the SQL commands are written as string literals in the backend program, and then passed to some intermediate interface or driver responsible for passing them to the database system, and retrieving the result.

This technique was very popular in the early days of backend and web development. However, it had several drawbacks. First, since the SQL statements in the backend are string literals, manipulating them was restricted to string operations, such as concatenation or template instatiations, which is somewhat low level. Second, writing SQL statements as string literals meant little IDE and toolchain support for them, it also is very error prone. Third, such a design posed several vulnerabilities, SQL commands were represented as string literals and assigned to variables, which by definition were used throughout the backend, and sometimes modified. Every access or modification to a variable like that posed an attack surface (or a potential bug) for hackers to abuse. A prime example of such vulnerability is [SQL-injection](https://www.owasp.org/index.php/SQL_Injection).

As backend development became increasingly important, its toolchains and frameworks received increased attention from the community. This led to the invention of ORMs. SQLalchemy is one of the most popular and mature ORMs available for python. ORMs embed SQL into their host language in a more sophisticated way. Developers can issue SQL commands without having to write SQL statements explicitly as strings. Instead, developers use typical constructs of the host language to represent the query logically, which is automatically translated into **safe** SQL statements via the ORM.

ORMs are generally a deep embedding. Constructs provided by the ORM (e.g. the capability to query data from a database) are represented as data types (e.g. classes and objects) in the host language, that are defined and provided by the ORM, which can be manipulated by the developers via the ORM's API, to issue SQL statements.

While SQL is based on the concepts of tables and relationship (e.g. foreign keys) between them, ORMs are based on classes and objects. An ORM allows developers to represent a table as a class in the host language. Rows in that table are then represented as objects that are instances of the corresponding class. A relationship between tables is represented via references between the associated objects.

For example, instead of using a statement like this:
```sql
SELECT user_id FROM users WHERE username='foo' AND password='bar';
```

An ORM supports achieving the same effect via class and objects in the host language:
```python
# User is a class representing the users table
User.query(User.user_id).filter(AND(User.username=='foo', User.password=='bar'))
```

# SQLAlchemy

In this course, we will focus specifically on SQLAlchemy as a sample ORM. SQLAlchemy is a terrific use case for studying embedded frameworks, since it uses a lot of the techniques we covered in class previously, so that it can achieve a friendly deep embedding into Python, that interfaces with SQL and databases. These notes follow closely this [great SQLAlchemy tutorial](https://docs.sqlalchemy.org/en/13/orm/tutorial.html).

The tutorial has more details about SQLAlchemy's capabilites and API, while the notes focus on providing a high level summary of the capabilities, and certain aspects that are interesting from an embedding point of view.

## Connecting to a Database

The first step of using SQLAlchemy is to connect it to the desired database system. This is achieved via creating an **engine**, which is the SQLAlchemy intermediate layer that sits between the developers python code, and the database system.

The engine is responsible for communicating with the database system, including sending and receiving network messages if the database is remote. The engine is also responsible for generating SQL queries that correspond to the developers code, using the syntax and features of the specific database system it is connected to. It is also responsible for parsing the results of queries into their corresponding python objects and values.

To create an engine, all you really need to do is provide SQLAlchemy with the address of the database to connect to. Below, we will connect to an in-memory sqlite database. This assumes that your computer has sqlite3 installed.

In [1]:
import sqlalchemy
from sqlalchemy import create_engine

# echo=True indicates that engine should print debugging information
# including all sql statements it issues
engine = create_engine('sqlite:///:memory:', echo=True)
print(engine, sqlalchemy.__version__)

Engine(sqlite:///:memory:) 1.3.11


# Schema Declaration via SQLAlchemy

The schema of a database specifies the structure of its tables. This includes the name of all tables, and each column inside them. This includes the name and type of each column (e.g. a number or a date or text), as well as any other properties or conditions of that column (e.g. marking a column as having only unique values).

Traditionally, a table can be defined in SQL via an insert statement. For example, this statement defines a table of authors, each with an ID a first name and a last name:
```SQL
CREATE TABLE authors(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL
);
```

Note that while the sytnax of SQL is standardized between all database systems (for most statements). Certain types and 
features are different or unique to specific database systems. For example, not all systems support AUTOINCREMENT.

## Declarative Base

We can define tables and schemas through SQLAlchemy via python constructs. Namely, we defined a table by defining a python class. Not every python class we define in an application will be a table. We can mark certain class as tables by inheriting from a special **declarative base** class that SQLAlchemy provides us.

First, we should create a declarative base via sqlalchemy's API. An application can have several declarative bases to support having several databases. We will go over how a declarative base is implemented in more detail in the next lecture.

In [2]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

## Defining Tables via Python Classes

Now, we can define all the tables we want as Python classes. We only have to **inherit from the declarative base**. These python classes are called **SQLAlchemy modules**.

Each module contains *static* attributes specifying the columns, their constraints, and other information about the table it represents. Additionally, since modules are just Python classes, they can also have Python-side attributes and functions. SQLAlchemy can automatically distinguish which are python-side and which are database side, by looking at the attributes and functions types.

Below, we will use SQLAlchemy declarative API to define several tables representing a library database.

In [3]:
from sqlalchemy import Column, Integer, String, ForeignKey
from sqlalchemy.types import Text, Boolean

class Author(Base):
    __tablename__ = 'authors' # the actual name in the DB

    id = Column(Integer, primary_key=True)
    first_name = Column(String, nullable=False)
    last_name = Column(String, nullable=False)

    def get_name(self): # python side
        return "{} {}".format(self.first_name, self.last_name)

    def __repr__(self): # python side
        return "<Author(id={}, name='{}'>".format(self.id, self.get_name())

Below, we define another table representing books in our library. An interesting attribute of books is that they have an author. In database terms, we have a one-to-many relationship between books and authors. An author can have many book, while a book can have one author (at least in our example).

We can define this relationship using SQLAlchemy's API in a standard way. First, we define a column that serves as a **foreign key** between the two tables. Second, we define a **logical relationship** between these two tables.

The logical relationship is a logical connection that SQLAlchemy automatically manages at the Python side. In particular, it provides a simple way to interact with rows in the two tables that are related via the foreign key. More on this later.

Notice that the definition of the relationship specifies three things:
1. The sqlalchemy module's name: This is specified as a string because the module may not have been defined yet at this point, and to support tables defined outside via the alternative classical mapping feature of SQLAlchemy, which defines tables without defining python classes.
2. The foreign keys: SQLAlchemy can deduce this automatically when only one foreign key exists between the two tables.
3. The backref name: this directs SQLAlchemy to define a back-reference relationship in the Author module, this is a useful shortcut to avoid having to define both ends of the relationship in both modules explicitly.

In [4]:
from sqlalchemy.orm import relationship

class Book(Base):
    __tablename__ = 'books'

    id = Column(Integer, primary_key=True)
    title = Column(String(75), nullable=False)
    synopsis = Column(Text, nullable=True)

    author_id = Column(Integer, ForeignKey(Author.id), nullable=False)
    author = relationship("Author", foreign_keys=[author_id], backref="books")

    def __repr__(self):
        return "<Book(id={}, title='{}', author='{}', has_synopsis={})>".format(
            self.id, self.title, self.author.get_name(), self.synopsis is not None)

Finally, we define a module for countries. We also specify a relationship between countries and authors. However, this is a many to many relationship. An author may have several nationalities, and a country can have many authors.

Traditionally, many-to-many relationships are defined via an association table, that lists related entries in both tables. We define this association table via the CountryAuthor module below.

We define the relationship inside the Country module. Note that we pass the name of the association module via the secondary parameter. This tells sqlalchemy that this is a many-to-many relation.

Notice that in the association module we only define foreign keys but no relations. This is only a matter of convinience. In our code, we only need to use the relationship via objects of countries and authors and never the association table.

In [5]:
from sqlalchemy import UniqueConstraint

class Country(Base):
    __tablename__ = 'country'

    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False, unique=True)
    code = Column(String(3), nullable=False, unique=True)
    
    authors = relationship("Author",
                           secondary="country_author",
                           backref="countries")

    def __repr__(self):
        return "<Country(id={}, name={}, code={})>".format(self.id, self.name,
                                                           self.code)

class CountryAuthor(Base):
    __tablename__ = 'country_author'
    
    author_id = Column(Integer, ForeignKey(Author.id), primary_key=True)
    country_id = Column(Integer, ForeignKey(Country.id), primary_key=True)
    native = Column(Boolean, nullable=False, default=True)
    
    #author = relationship("Author")
    #country = relationship("Country")

    def __repr__(self):
        return "<CountryAuthor(author_id={}, country_id={}, native={})".format(
            self.author_id, self.country_id, self.native)

## Creating the Tables

Note that we have only defined python classes so far. The database itself has not been modified yet, and no "physical" tables were created. We can tell SQLAlchemy to create the tables for us via the declarative base. When creating the tables, we must specify a corresponding engine, which will be used to generate and issue the corresponding SQL commands.

Note how this call generates many SQL statements, corresponding to each of the modules we defined above.

In [6]:
Base.metadata.create_all(engine)

2019-11-14 00:20:03,948 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-11-14 00:20:03,950 INFO sqlalchemy.engine.base.Engine ()
2019-11-14 00:20:03,953 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-11-14 00:20:03,957 INFO sqlalchemy.engine.base.Engine ()
2019-11-14 00:20:03,960 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("country")
2019-11-14 00:20:03,964 INFO sqlalchemy.engine.base.Engine ()
2019-11-14 00:20:03,967 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("country")
2019-11-14 00:20:03,970 INFO sqlalchemy.engine.base.Engine ()
2019-11-14 00:20:03,972 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("books")
2019-11-14 00:20:03,975 INFO sqlalchemy.engine.base.Engine ()
2019-11-14 00:20:03,976 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("books")
2019-11-14 00:20:03,979 INFO sqlalchemy.engine.base.Engine ()
2019-11-14 00:20:03,98

# Data Manipulation

Now that we have our tables defined. It is time to add some data to them.

SQLAlchemy supports data manipulation via regular python manipulation of objects. This includes inserting, updating, and deleting of data.

## Session and Transactions

SQLAlchemy supports transactions in the ACID model. A transaction consists of several commands, which may include data manipulation, queries, and even schema manipulation. A transaction is initiated before all of its commands, and is then populated with commands one at a time. A transaction can be committed, at which point its commands are committed to the database. Transactions can also be rolled back (e.g. in the case of errors), at which point their effects are undone.

ACID stands for:
1. Atomicity: Each transaction is atomic. Either all of its commands are committed, or all of them fail.
2. Consistency: If a transaction is committed, then all of it commands are consistent with the database constraints (e.g. it never sets the value of a column to null if the column is not allowed to be null).
3. Isolation: changes made by a transaction do not affect other concurrent transaction, unless that transaction was committed, or the changes were asked to be flushed explicitly.
4. Durable: committed transaction are indeed committed in the database physically, and they will survive future errors or crashes.

A transaction in SQLAlchemy is implemented as part of the session abstraction. A session is an SQLAlchemy object that can be populated with commands and manipulation. The sessions provides a **commit** and **rollback** functions that can be used to manipulate the underlying transaction.

Additionally, the session provides a **flush** function, which can be used to flush the changes made so far to the database, so that their effects are visible to other transactions and queries. Even if some commands are flushed successfully, future commands in the same transaction may cause errors, which will cause the entire transaction to rollback, including the flushed commands. This is not the case for **committed** commands, which will never get rolled back.

Sessions are created via a session maker. A session maker can be configured initially by providing an engine, and then used to create new sessions. This is shown below. Note that this is a simple [factory pattern](https://en.wikipedia.org/wiki/Factory_method_pattern).

Note that SQLAlchemy supports several optional parameters that can be provided at the SessionMaker or the session level. Most noteably autoflush and autocommit, which default to True and False respectively, which specify whether any given command is immediately flushed or committed. A session with autocommit=False can be committed explicitly by calling .commit().

The transaction/session API of SQLAlchemy is very extensive, and include a variety of configurable parameters, as well as support for manual management of transactions, subtransactions, and other customizable behavior. For more details, look at the [sqlalchemy session docs](https://docs.sqlalchemy.org/en/13/orm/session_api.html).

In [7]:
from sqlalchemy.orm import sessionmaker
SessionMaker = sessionmaker(bind=engine)
# SessionMaker can now give us sessions

## Inserting Elements

Now that we have the ability to create sessions. We can start inserting rows into our database. This is done via three steps, the last of which may be implicit:

1. A new instance object of the module representing the desired table is created. The object can be provided all the content data via its constructors (which is inherited from the declartive base), or by manipulating its attributes explicitly.
2. The new instances are added to a session/transaction. This does not commit them to the database directly (unless autocommit=True). However, it may flush them to the database if autoflush=True.
3. The session is committed via .commit() or implicitly when autocommit=true.

Below, we create a new author and country object. We also indicate that they are related by manipulating their logical relation.

In [8]:
# Create session
session = SessionMaker(autoflush=False)

# Create new objects
author = Author(first_name='J. K.', last_name='Rowling')
country = Country()
country.name = 'United Kingdom'
country.code = 'UK'

# Add objects to session
session.add(author)
session.add(country)

# Notice that the objects are not inserted to the database
# but they are marked as new by the session
print(session.new)

# Country.authors is a many-to-many relation we defined above
# also Author.countries is its backref relation
# because it is many-to-many, both directions of the relation are lists.
author.countries.append(country)
#country.authors.append(author) # this is also fine



IdentitySet([<Author(id=None, name='J. K. Rowling'>, <Country(id=None, name=United Kingdom, code=UK)>])


In [9]:
# We can commit the session
session.commit()

2019-11-14 00:20:04,190 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-14 00:20:04,194 INFO sqlalchemy.engine.base.Engine INSERT INTO authors (first_name, last_name) VALUES (?, ?)
2019-11-14 00:20:04,197 INFO sqlalchemy.engine.base.Engine ('J. K.', 'Rowling')
2019-11-14 00:20:04,203 INFO sqlalchemy.engine.base.Engine INSERT INTO country (name, code) VALUES (?, ?)
2019-11-14 00:20:04,206 INFO sqlalchemy.engine.base.Engine ('United Kingdom', 'UK')
2019-11-14 00:20:04,210 INFO sqlalchemy.engine.base.Engine INSERT INTO country_author (author_id, country_id, native) VALUES (?, ?, ?)
2019-11-14 00:20:04,212 INFO sqlalchemy.engine.base.Engine (1, 1, 1)
2019-11-14 00:20:04,218 INFO sqlalchemy.engine.base.Engine COMMIT


## Rollbacks

The previous transaction was successfully committed. It does not violate any constraints set by our database.

The following transaction will fail. In particular, because we are inserting a new country with the same code as the UK, which violates the uniqueness constraint defined in the Country module above.

In [10]:
session = SessionMaker()

author = Author(first_name='Other', last_name='Author')
country = Country(name='Other Country', code='UK')

session.add_all([author, country])
try:
    session.commit()
except Exception as e:
    print('')
    print(e)
    # session.rollback() # the transaction is automatically rolledback

2019-11-14 00:20:04,336 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-14 00:20:04,338 INFO sqlalchemy.engine.base.Engine INSERT INTO authors (first_name, last_name) VALUES (?, ?)
2019-11-14 00:20:04,339 INFO sqlalchemy.engine.base.Engine ('Other', 'Author')
2019-11-14 00:20:04,343 INFO sqlalchemy.engine.base.Engine INSERT INTO country (name, code) VALUES (?, ?)
2019-11-14 00:20:04,344 INFO sqlalchemy.engine.base.Engine ('Other Country', 'UK')
2019-11-14 00:20:04,345 INFO sqlalchemy.engine.base.Engine ROLLBACK

(sqlite3.IntegrityError) UNIQUE constraint failed: country.code
[SQL: INSERT INTO country (name, code) VALUES (?, ?)]
[parameters: ('Other Country', 'UK')]
(Background on this error at: http://sqlalche.me/e/gkpj)


# Data Query

SQLAlchemy supports querying tables via its [Query API](https://docs.sqlalchemy.org/en/13/orm/query.html) and Query objects. The Query API includes all standard SQL query constructs, including filter, group by, order, and others. It also supports eager and lazy joins (more on that later), as well as a variety of aggregators.

The query API constructs are used in the python side and then transformed into SQL statements that get executed at the database side. This is important for efficiency. One can achieve a similar functionality to a filter query by retrieving all the rows, and performing the filter in python via a for-loop and an if statement. However, this will be much slower:
1. It will transfer all the data from the database to python, this is particular slow if the database is remote and requires network access.
2. It will execute the filter in python, which is a slow interpreted language, as opposed to compiled and optimized binaries of the database system (usually C/C++).
3. Database systems are optimized to perform many filters quickly, via index and caches and other techniques.

Queries are lazy. Creating a query does not immediately execute it. This is because a query can be modified after creation several times (e.g. by calling the filter function). The query is executed when the developer indicate that it has been finalized, by calling .all(), .first(), .get(), or certain final aggregators such as .count()

Query results can be a python object (instances of queries modules) or a list containing such objects, depending on whether get/first or all was used.

These objects contain the data in the corresponding rows inside their attributes. This includes other rows that are related to them via a relation. These attributes and rows can be accessed directly by accessing the properites of the python objects.

In [11]:
# create new session
session = SessionMaker()

# Get the country corresponding to country code 'uk'
query1 = session.query(Country)
query1 = query1.filter(Country.code=='UK')
query2 = session.query(Author)

print('queries created but not executed yet!\n')

uk = query1.first()
all_authors = query2.all()

print('\nqueries executed!')

print(all_authors)
print(uk)

queries created but not executed yet!

2019-11-14 00:20:04,445 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-14 00:20:04,448 INFO sqlalchemy.engine.base.Engine SELECT country.id AS country_id, country.name AS country_name, country.code AS country_code 
FROM country 
WHERE country.code = ?
 LIMIT ? OFFSET ?
2019-11-14 00:20:04,450 INFO sqlalchemy.engine.base.Engine ('UK', 1, 0)
2019-11-14 00:20:04,456 INFO sqlalchemy.engine.base.Engine SELECT authors.id AS authors_id, authors.first_name AS authors_first_name, authors.last_name AS authors_last_name 
FROM authors
2019-11-14 00:20:04,457 INFO sqlalchemy.engine.base.Engine ()

queries executed!
[<Author(id=1, name='J. K. Rowling'>]
<Country(id=1, name=United Kingdom, code=UK)>


## Lazy vs Eager Joins

By default, relations are lazy. A query retrieves the data of the queried rows only. The data of other rows that are related is retrieved on demand, when that relationship is accessed.

For example, below we can see that plain attributes of the row corresponding to the UK are immediately available. However, accessing the authors relationship issues an SQL query that fetches it on-demand, before returning the entries of that relationship.

Note that future use of the relationship do not cause other SQL queries, since the relationship content gets cached after the first use.

In [12]:
print('plain attributes are available')
print('name = ', uk.name)
print('code = ', uk.code)

print('\nRelationship are lazy, below you will see the SQL query performing the join\n')
print('\nauthors = ', uk.authors)

print('\nRelationship is cached after first use')
print('authors = ', uk.authors)

plain attributes are available
name =  United Kingdom
code =  UK

Relationship are lazy, below you will see the SQL query performing the join

2019-11-14 00:20:04,604 INFO sqlalchemy.engine.base.Engine SELECT authors.id AS authors_id, authors.first_name AS authors_first_name, authors.last_name AS authors_last_name 
FROM authors, country_author 
WHERE ? = country_author.country_id AND authors.id = country_author.author_id
2019-11-14 00:20:04,605 INFO sqlalchemy.engine.base.Engine (1,)

authors =  [<Author(id=1, name='J. K. Rowling'>]

Relationship is cached after first use
authors =  [<Author(id=1, name='J. K. Rowling'>]


When the query is defined, it can be configured to load certain relations eagerly as part of the initial SQL query. The join strategy (lazy or eager) can also be specified in the relationship definition at the module, which sets the default behavior for that relation.

Eager loading is good for cases when we know that the relation will be accessed on many of the resulting objects. Since that will allow us to perform a single SQL query only, instead of several ones. Otherwise, lazy join is recommended.

In [13]:
from sqlalchemy.orm import joinedload

print('The following query will have a join in it eagerly')
uk = session.query(Country).options(joinedload(Country.authors)).filter(Country.code=='UK').first()

print('\nNow authors will be ready immediately')
print(uk, uk.authors)

The following query will have a join in it eagerly
2019-11-14 00:20:04,708 INFO sqlalchemy.engine.base.Engine SELECT anon_1.country_id AS anon_1_country_id, anon_1.country_name AS anon_1_country_name, anon_1.country_code AS anon_1_country_code, authors_1.id AS authors_1_id, authors_1.first_name AS authors_1_first_name, authors_1.last_name AS authors_1_last_name 
FROM (SELECT country.id AS country_id, country.name AS country_name, country.code AS country_code 
FROM country 
WHERE country.code = ?
 LIMIT ? OFFSET ?) AS anon_1 LEFT OUTER JOIN (country_author AS country_author_1 JOIN authors AS authors_1 ON authors_1.id = country_author_1.author_id) ON anon_1.country_id = country_author_1.country_id
2019-11-14 00:20:04,709 INFO sqlalchemy.engine.base.Engine ('UK', 1, 0)

Now authors will be ready immediately
<Country(id=1, name=United Kingdom, code=UK)> [<Author(id=1, name='J. K. Rowling'>]


## Transactions and Queries

A transaction may include some data manipulation as well as queries. Additionally, there may be concurrent transactions performing queries as well.

This raises a question of isolation. Should the effect of a pending (a transaction that has not been committed yet) be visible to queries made in that transaction, or other concurrent transactions?

SQLAlchemy manages the isolation via flushes. All non-flushed changes are not visible to queries of any transaction. Changes that are flushed successfuly, are visible to other transaction. However, they may get rolled back if their issuing transaction fails. Remember, sessions autoflush changes by default.

In [14]:
session1 = SessionMaker(autoflush=False)
session2 = SessionMaker(autoflush=False)

country = Country(name='United States', code='US')
session1.add(country)

print('US was added to session 1, but not flushed')
print('US will not be visibile in either sessions!\n')
countries1 = session1.query(Country).all()
countries2 = session2.query(Country).all()
print('\nsession1 -->', countries1)
print('session2 -->', countries2, '\n')

print('Flush issued! (notice there is no COMMIT below)')
session1.flush()

print('\nUS is flushed, now it will become visible!\n')
countries1 = session1.query(Country).all()
countries2 = session2.query(Country).all()
print('\nsession1 -->', countries1)
print('session2 -->', countries2, '\n')

print('Commit session 1')
session1.commit()

US was added to session 1, but not flushed
US will not be visibile in either sessions!

2019-11-14 00:20:04,793 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-14 00:20:04,794 INFO sqlalchemy.engine.base.Engine SELECT country.id AS country_id, country.name AS country_name, country.code AS country_code 
FROM country
2019-11-14 00:20:04,795 INFO sqlalchemy.engine.base.Engine ()
2019-11-14 00:20:04,797 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-14 00:20:04,798 INFO sqlalchemy.engine.base.Engine SELECT country.id AS country_id, country.name AS country_name, country.code AS country_code 
FROM country
2019-11-14 00:20:04,798 INFO sqlalchemy.engine.base.Engine ()

session1 --> [<Country(id=1, name=United Kingdom, code=UK)>]
session2 --> [<Country(id=1, name=United Kingdom, code=UK)>] 

Flush issued! (notice there is no COMMIT below)
2019-11-14 00:20:04,801 INFO sqlalchemy.engine.base.Engine INSERT INTO country (name, code) VALUES (?, ?)
2019-11-14 00:20:04,802 INF

## Updating a Row

After an object has been retrieved via a query, its data can be modified via regular python assignments to its attributes. This does not issue the update to the database yet. Instead, the update object must be added to a session, and that session has to be committed (explicitly or implicitly) for the update to get committed to the database.

The session can also be flushed, to make the update visible without committing it.

Update objects are marked by the session as "dirty".

In [15]:
session = SessionMaker()

# get the country with primary key (in this case id) = 2
usa = session.query(Country).get(2)

# print usa to the screen
print('')
print(usa)
print('')

# update its name
usa.name = 'United State of America'
session.add(usa)

print('new = ', session.new)
print('dirty = ', session.dirty, '\n')

# commit
session.commit()
'''

'''
print('')

2019-11-14 00:20:04,954 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-14 00:20:04,957 INFO sqlalchemy.engine.base.Engine SELECT country.id AS country_id, country.name AS country_name, country.code AS country_code 
FROM country 
WHERE country.id = ?
2019-11-14 00:20:04,959 INFO sqlalchemy.engine.base.Engine (2,)

<Country(id=2, name=United States, code=US)>

new =  IdentitySet([])
dirty =  IdentitySet([<Country(id=2, name=United State of America, code=US)>]) 

2019-11-14 00:20:04,971 INFO sqlalchemy.engine.base.Engine UPDATE country SET name=? WHERE country.id = ?
2019-11-14 00:20:04,973 INFO sqlalchemy.engine.base.Engine ('United State of America', 2)
2019-11-14 00:20:04,976 INFO sqlalchemy.engine.base.Engine COMMIT



Relations can also be updated similarly. One-to-many relations are updated via updating an existing object relation. Many-to-Many relations may be updated via creating a new instance of the secondary/association module, or via the relation directly.

In [16]:
# Make J. K. Rowling american! (just cause I can)
rowling = session.query(Author).first()
rowling.countries.append(usa)

# Alternative way: this allows setting data on the association object
# rowling_usa = CountryAuthor(author_id=rowling.id, country_id=usa.id, native=False)
# session.add(rowling_usa)

session.commit()

2019-11-14 00:20:05,068 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-14 00:20:05,069 INFO sqlalchemy.engine.base.Engine SELECT authors.id AS authors_id, authors.first_name AS authors_first_name, authors.last_name AS authors_last_name 
FROM authors
 LIMIT ? OFFSET ?
2019-11-14 00:20:05,069 INFO sqlalchemy.engine.base.Engine (1, 0)
2019-11-14 00:20:05,071 INFO sqlalchemy.engine.base.Engine SELECT country.id AS country_id, country.name AS country_name, country.code AS country_code 
FROM country, country_author 
WHERE ? = country_author.author_id AND country.id = country_author.country_id
2019-11-14 00:20:05,071 INFO sqlalchemy.engine.base.Engine (1,)
2019-11-14 00:20:05,073 INFO sqlalchemy.engine.base.Engine SELECT country.id AS country_id, country.name AS country_name, country.code AS country_code 
FROM country 
WHERE country.id = ?
2019-11-14 00:20:05,074 INFO sqlalchemy.engine.base.Engine (2,)
2019-11-14 00:20:05,074 INFO sqlalchemy.engine.base.Engine INSERT INTO country